In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import os, time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 연산을 수행할 device 설정

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# 먼저, 기본적인 CNN layer의 연산 원리를 봅시다.

In [3]:
cnn = nn.Conv2d(in_channels=1, 
                out_channels=3, 
                kernel_size=2, 
                stride=1, bias=False)


In [4]:
print(cnn.weight)
print(cnn.weight.shape)

Parameter containing:
tensor([[[[-0.2565,  0.1148],
          [-0.1213,  0.1491]]],


        [[[-0.4319, -0.0354],
          [-0.4376, -0.4088]]],


        [[[ 0.3053,  0.4963],
          [ 0.1610,  0.3246]]]], requires_grad=True)
torch.Size([3, 1, 2, 2])


In [5]:
new_weight = torch.ones((3,1,2,2), dtype=float)

new_weight[0] *= 1
new_weight[1] *= 2
new_weight[2] *= 3

print(new_weight)

tensor([[[[1., 1.],
          [1., 1.]]],


        [[[2., 2.],
          [2., 2.]]],


        [[[3., 3.],
          [3., 3.]]]], dtype=torch.float64)


In [6]:
cnn.weight = nn.Parameter(new_weight)
print(cnn.weight)

Parameter containing:
tensor([[[[1., 1.],
          [1., 1.]]],


        [[[2., 2.],
          [2., 2.]]],


        [[[3., 3.],
          [3., 3.]]]], dtype=torch.float64, requires_grad=True)


In [7]:
x = torch.ones((2, 1, 5, 5), dtype=float) # data수는 2, data channel은 1, (height, width) = (5,5)인 데이터

In [8]:
result = cnn(x)
print(result)

tensor([[[[ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.]],

         [[ 8.,  8.,  8.,  8.],
          [ 8.,  8.,  8.,  8.],
          [ 8.,  8.,  8.,  8.],
          [ 8.,  8.,  8.,  8.]],

         [[12., 12., 12., 12.],
          [12., 12., 12., 12.],
          [12., 12., 12., 12.],
          [12., 12., 12., 12.]]],


        [[[ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.]],

         [[ 8.,  8.,  8.,  8.],
          [ 8.,  8.,  8.,  8.],
          [ 8.,  8.,  8.,  8.],
          [ 8.,  8.,  8.,  8.]],

         [[12., 12., 12., 12.],
          [12., 12., 12., 12.],
          [12., 12., 12., 12.],
          [12., 12., 12., 12.]]]], dtype=torch.float64,
       grad_fn=<ConvolutionBackward0>)


## 1. 네트워크 구조 & Input Flow 작성하기
- <span style = 'font-size:1.2em;line-height:1.5em'>Input(28*28) -> 1st convolution(+relu) + MaxPooling -> 2nd convolution(+relu) + Maxpooling -> (Flatten) -> 1st FFNN(+relu) -> 2nd FFNN </span>

    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1st conv</b></span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>in_channel=1, out_channel=64: </b>1개 channel을 갖는 image를 받아 64개의 channel을 생성</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>kernel_size=(3,3): </b>채널의 각 위치의 값을 계산하는 kernel의 크기는 height 3, width 3. 이러한 kernel이 1*64개만큼 생성됨</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>stride=(1,1), padding=1: </b>kernel의 이동은 세로로 1, 가로로 1만큼. conv층에 들어오는 데이터에 padding을 양옆에 1만큼 해줌</span>

    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1st MaxPooling</b></span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>kernel_size=(2,2): </b>kernel의 크기는 height 2, width 2. 이러한 kernel이 conv layer의 out_channel개만큼 생성됨</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>stride=(2,2): </b>kernel의 이동은 세로로 2, 가로로 2만큼.</span>
        
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>2st conv</b></span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>in_channel=64, out_channel=128: </b>64개 channel을 갖는 1st MaxPooling 층의 output값을 받아 128개의 channel을 생성</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>kernel_size=(3,3): </b>채널의 각 위치의 값을 계산하는 kernel의 크기는 height 3, width 3. 이러한 kernel이 64*128개만큼 생성됨</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>stride=(1,1), padding=1: </b>kernel의 이동은 세로로 1, 가로로 1만큼. conv층에 들어오는 데이터에 padding을 양옆에 1만큼 해줌</span>

    - <span style = 'font-size:1.1em;line-height:1.5em'><b>2nd MaxPooling</b></span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>kernel_size=(2,2): </b>kernel의 크기는 height 2, width 2. 이러한 kernel이 conv layer의 out_channel개만큼 생성됨</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>stride=(2,2): </b>kernel의 이동은 세로로 2, 가로로 2만큼.</span>
        
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Flatten</b></span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>x.reshape(): </b>FFNN의 입력값으로 넣을 수 있도록, 2nd Pooling Layer의 결과 값인 (batch_size, 128, 7, 7) 크기의 tensor를 (batch_size, 128\*7\*7)크기의 tensor로 형태 변환</span>
        
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1st FFNN, 2nd FFNN</b></span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>1st FFNN: </b>7\*7\*128차원의 벡터를 100차원의 벡터로 보내는 FFNN (with relu 활성화 함수)</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'><b>2nd FFNN: </b>100차원의 벡터를 10차원(=class 수)의 벡터로 보내는 FFNN (원래는 softmax함수를 함께 써야 하지만, Pytorc)</span>
            - <span style = 'font-size:0.9em;line-height:1.5em'>원래는 softmax함수를 함께 써야 하지만, Pytorch의 cross_entropy함수의 특성상, softmax를 생략</span>

In [9]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=(1,1), padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.fc1   = nn.Linear(7*7*128, 100, bias=True)
        self.fc2   = nn.Linear(100, 10, bias=True)
        self.apply(self._init_weights)
        
    def _init_weights(self, submodule):
        if isinstance(submodule, nn.Conv2d):
            nn.init.xavier_normal_(submodule.weight)
            if submodule.bias is not None:
                submodule.bias.data.fill_(0.01)
        if isinstance(submodule, nn.Linear): # submodule이 nn.Linear에서 생성된 객체(혹은 인스턴스이면)
            nn.init.kaiming_normal_(submodule.weight) #해당 submodule의 weight는 He Initialization으로 초기화
            if submodule.bias is not None:
                submodule.bias.data.fill_(0.01) # 해당 submodule의 bias는 0.01로 초기화
                
    def forward(self, x):
        # (n_data, n_channel, height, width)으로 연산 결과의 크기 표기
        # 1st conv layer
        out = self.conv1(x) # shape: (batch,1,28,28) -> (batch,32,28,28)
        out = F.relu(out) 
        out = self.pool1(out) # (batch,32,28,28) -> (batch,32,14,14)
        
        # 2nd conv layer
        out = self.conv2(out) # (batch,32,14,14) -> (batch,64,14,14)
        out = F.relu(out)
        out = self.pool2(out) # (batch,64,14,14) -> (batch,64,7,7)
        
        # Flatten
        out = out.reshape(-1, 7*7*128)
        
        # 1st FFNN
        out = self.fc1(out)
        out = F.relu(out)
        
        # 2nd FFNN
        out = self.fc2(out)
        return out

## 2. pytorch 모델을 학습하기 위한 데이터 셋 생성하기 
### (By datasets, transforms (in torchvision))
- <span style = 'font-size:1.2em;line-height:1.5em'><b>1.</b> 데이터를 다운받을 디렉토리 선언 (optional)</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>2.</b> 데이터를 변환할 방법 선언(예시: numpy array 형태의 데이터를 torch Tensor로</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>torch 모델에 입력하는 데이터도 반드시 torch tensor여야 합니다.</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>3.</b> 데이터 셋 생성(다운로드 여부, 학습/평가 데이터 여부, 데이터 변환 방법 등)</span>

In [10]:
data_path = 'data'
if not os.path.exists(data_path):
    os.makedirs(data_path)
    
transform = transforms.Compose([transforms.ToTensor(), # 이미지를 텐서로 변경하고
                                transforms.Normalize((0.1307,),(0.3081,))  # 이미지를 0.1307, 0.3081값으로 normalize (나중에 봅시다.)
                               ])

trn_dset = datasets.MNIST(root=data_path, train=True, transform=transform, download=True)
tst_dset = datasets.MNIST(root=data_path, train=False, transform=transform, download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



## 3. Mini-batch 데이터를 자동으로 생성해주는 DataLoader 생성하기
### (By DataLoader class (in torch.utils.data))

In [11]:
from torch.utils.data import DataLoader

BATCH_SIZE = 2**9
trn_loader = DataLoader(trn_dset, batch_size = BATCH_SIZE, shuffle=True, drop_last=False)
tst_loader = DataLoader(tst_dset, batch_size = BATCH_SIZE, shuffle=False, drop_last=False)

## 4. 모델 객체를 생성하고 이 모델을 GPU에서 사용할지 GPU에서 사용할지 결정

In [12]:
model = MyNet()
model = model.to(device)

## 5. Optimizer 설정하기 (Create an optimizer)

In [13]:
import torch.optim as optim

my_opt = optim.Adam(params = model.parameters(), lr = 2e-4)

## 6. loss function 설정하기

In [14]:
loss_func = nn.CrossEntropyLoss(reduction='sum')

## 7. 매 Epoch에 드는 시간 측정

In [16]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## 8. 학습을 해봅시다.

### 1. Train the model
- <span style = 'font-size:1.2em;line-height:1.5em'>Mini-batch train data에 대해서 다음과 같은 과정을 수행합니다. (모든 train data를 다 입력할 때 까지 = 1 epoch)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1-(1).</b> mini-batch x,y를 모델에 입력합니다. `x.shape = (n_data,1,28,28)`, `y.shape=(n_data)`. 동시에 모델이 gpu에 있으면 data도 gpu로 올려줍니다.</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'> <b>n_data: </b>mini-batch data수, <b>1: </b>channel수(흑백이라서 단일 채널. 칼라 이미지(RGB)는 기본으로 3으로 설정됨) <b>28: </b>Width, <b>28: </b>height</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1-(2).</b> 이전 단계에서 계산되어 남아있는 optimizer의 gradient 값들을 전부 0으로 비워줍니다. (안그럼 계속 누적되어서 계산됩니다!)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1-(3).</b> Forward Propagation: Mini-batch 데이터를 모델에 입력하여 최종 output값을 계산하는 forward propagation을 진행합니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1-(4).</b> Loss Calculation: 실제 y와 모델이 예측한 y사이의 loss를 구합니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1-(5).</b> Gradient Calculation: Loss로부터 모델의 모든 parameter의 gradient를 계산합니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1-(6).</b> BackPropgation: 계산된 gradient를 활용하여 각 parameter값을 update합니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>1-(7).</b> trn_loss에 minibatch loss를 누적해서 계산하기</span>

### 2. Evaluate the model (for validation)
- <span style = 'font-size:1.2em;line-height:1.5em'>Mini-batch validation data에 대해서 다음과 같은 과정을 수행합니다. (단, validation에서는 학습을 진행하면 안되기 때문에, `torch.no_grad()`와 `model.eval()`가 반드시 필요)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>2-(1).</b> mini-batch x,y를 모델에 입력합니다. `x.shape = (n_data,1,28,28)`, `y.shape=(n_data)`. 동시에 모델이 gpu에 있으면 data도 gpu로 올려줍니다.</span>
        - <span style = 'font-size:1.0em;line-height:1.5em'> <b>n_data: </b>mini-batch data수, <b>1: </b>channel수(흑백이라서 단일 채널. 칼라 이미지(RGB)는 기본으로 3으로 설정됨) <b>28: </b>Width, <b>28: </b>height</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>2-(2).</b> Forward Propagation: Mini-batch validation 데이터를 모델에 입력하여 최종 output값을 계산하는 forward propagation을 진행합니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>2-(3).</b> 매 epoch마다 validation set에서 성능이 어떻게 변하는지 모니터링 하기 위해 모델이 예측한 결과와 loss를 계산합니다. 그리고 그 결과를 계속해서 누적합니다.</span>

### 3. 전체 결과 출력
- <span style = 'font-size:1.2em;line-height:1.5em'>Mini-batch별 예측 결과와 정답을 누적하여 전체 예측 결과와 전체 정답을 만든 뒤, mean test error와 accuracy를 산출</span>

In [17]:
# 전체 데이터를 n_epoch(10)번 반복하여 넣을 때 까지 학습합니다.
n_epochs = 10

# 매 epoch마다 반복
for epoch in range(n_epochs):
    start_time = time.time()
    model.train()
    trn_loss = 0
    # 매 mini-batch train data마다 반복
    for i, (x, y) in enumerate(trn_loader):
        # 1-(1): 모델에 입력하기 위해서 데이터의 형태 변환
        x = x.to(device) # x.shape: (batch_size,1, 28,28)
        y = y.to(device)
        
        # 1-(2): 기존에 계산된 gradient를 0으로 reset
        my_opt.zero_grad()
        
        # 1-(3): Forward Propagation
        y_pred_prob = model(x)
        
        # 1-(4): Loss Calculation
        loss = loss_func(y_pred_prob, y)
        
        # 1-(5): Gradient Calculation(Backprop)
        loss.backward()
        
        # 1-(6): Update parameter
        my_opt.step()
        
        # 1-(7): trn_loss에 mini_batch loss를 누적해서 계산하기
        trn_loss += loss.item()
        
    trn_loss /= len(trn_loader.dataset)
    
    model.eval()
    results_pred = []
    results_real = []
    val_loss = 0
    with torch.no_grad():
        # 매 mini-batch validation data마다 반복
        for i, (x, y) in enumerate(tst_loader):
            # 2-(1)
            x = x.to(device) # x.shape: (batch_size,1,28,28)
            y = y.to(device)
            
            # 2-(2)
            y_pred_prob = model(x)
            y_pred_label = np.argmax(y_pred_prob, axis=1)

            # 2-(3)
            loss = loss_func(y_pred_prob, y)
            val_loss += loss
            
            results_pred.extend(y_pred_label.cpu().detach().numpy())
            results_real.extend(y.cpu().detach().numpy())
            
        # 3.
        val_loss /= len(tst_loader.dataset)
        results_pred = np.array(results_pred)
        results_real = np.array(results_real)
        accuracy = np.sum(results_pred == results_real) / len(tst_loader.dataset)
        
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {trn_loss:.3f} | Test Loss: {val_loss:.3f} | Test Acc: {100*accuracy:.3f}% ')

Epoch: 01 | Time: 1m 43s
	Train Loss: 0.140 | Test Loss: 0.093 | Test Acc: 97.460% 
Epoch: 02 | Time: 1m 43s
	Train Loss: 0.087 | Test Loss: 0.071 | Test Acc: 97.830% 
Epoch: 03 | Time: 1m 43s
	Train Loss: 0.067 | Test Loss: 0.052 | Test Acc: 98.400% 
Epoch: 04 | Time: 1m 43s
	Train Loss: 0.055 | Test Loss: 0.049 | Test Acc: 98.380% 
Epoch: 05 | Time: 1m 44s
	Train Loss: 0.048 | Test Loss: 0.039 | Test Acc: 98.720% 
Epoch: 06 | Time: 1m 44s
	Train Loss: 0.041 | Test Loss: 0.038 | Test Acc: 98.740% 
Epoch: 07 | Time: 1m 44s
	Train Loss: 0.037 | Test Loss: 0.038 | Test Acc: 98.800% 
Epoch: 08 | Time: 1m 43s
	Train Loss: 0.034 | Test Loss: 0.034 | Test Acc: 98.790% 
Epoch: 09 | Time: 1m 43s
	Train Loss: 0.031 | Test Loss: 0.038 | Test Acc: 98.690% 
Epoch: 10 | Time: 1m 44s
	Train Loss: 0.028 | Test Loss: 0.036 | Test Acc: 98.720% 
